In [14]:
import pandas as pd
import glob
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.width", 2000)

In [15]:
import numpy as np
np.set_printoptions(precision=4,suppress=True)
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
from tqdm import tqdm

In [17]:
frames = []
for f in glob.glob('sample_with_header.csv'):
    frames.append(pd.read_csv(f))

In [18]:
raw_df = pd.concat(frames)

In [19]:
raw_df

,SichterGruppe,SichterName,Host,kind,Datetime,Content,Url,Authorid,Tag,entryid,Id,TwistId,TopicId,TopicName,IsInitialEntry,IsPrivate
0,NaN,NaN,twitter.com,1,21.10.2016 23:18:00,HighTechno ! Ich will Claras WG mal so richtig zu #Elektro dancen sehen . https://t.co/oLKYDGHBGQ,https://twitter.com/Carina42008/status/789576493810737152#aid=1394595750,3796107.0,NaN,bc6c4c70-a00d-429d-8bde-4e7afa072a15,31796013,04b9f324-5cee-4dd3-8387-0ff934ec1472,551631,Alle,False,False
1,NaN,NaN,twitter.com,3,21.10.2016 23:18:00,"Yeah , der WG gefällt deine Musik . Hol dir auch superschnelles Internet ! https://t.co/0dPArCUv0b https://t.co/HHRZaAQnte http://pbs.twimg.com/amplify_video_thumb/0123456789001234567890/img/QEQntlwB_ifYH3Gv.jpg",https://twitter.com/telekomerleben/status/789576496071446528#aid=20459730,1060906.0,NaN,5a676fbd-f7c7-45b1-8744-3a7fd8ebc9e9,31795989,04b9f324-5cee-4dd3-8387-0ff934ec1472,551631,Alle,True,False
2,Sichter FB/TW,Hotspot,www.facebook.com,1,24.09.2016 14:57:02,"Muss ich mit Hotspot registrieren wlan to go auf meinem Handy zu gehen , weiß ich nicht , wie es zu tun ich bin Telekom Kunde",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611&uid=122768271103611_1141448189235609&aid=992943974115174&vid=2&wid=122768271103611,7707299.0,NaN,94e8d4f1-60c1-45e5-894a-c11a3ff7b655,30744441,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,True,False
3,NaN,NaN,www.facebook.com,3,24.09.2016 15:12:35,"Hallo Fiona . Wenn Sie für WLAN TO GO angemeldet sind , dann können Sie unsere HotSpots nutzen in dem Sie sich mit Ihrer T-Online E-Mail-Adresse und dem dazugehörigen Passwort ( das gleiche wie für das Kundencenter ) anmelden . Viele Grüße Christian We .",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141458792567882&cpid=&aid=122768271103611&vid=2&wid=122768271103611,254017.0,NaN,263da5f3-4f0c-4a3e-ac27-358eb033912f,30744872,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
4,NaN,NaN,www.facebook.com,4,24.09.2016 15:16:45,"ich bin nicht gemeldet , es ok ich vergesse es",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141461172567644&cpid=1141448189235609_1141458792567882&aid=992943974115174&vid=2&wid=122768271103611,7707299.0,NaN,51a8de05-48e4-4422-822f-50aba72fb5f5,30745002,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
5,Sichter FB/TW,Hotspot,www.facebook.com,4,24.09.2016 15:18:01,"Jetzt muss ich mal eins fragen . Ich kann doch nicht mit Wlan to go alle Hotspots der Telekom kostenfrei nutzen , oder ? Was ich weiss nur die Hotspots der teilnehmenden Partner / Telekomkunden der Fa . Fon",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141461895900905&cpid=&aid=1096037533756491&vid=2&wid=122768271103611,127469.0,NaN,a255ed96-818f-42b2-84f1-0c0c7e2581da,30745045,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
6,NaN,NaN,www.facebook.com,4,24.09.2016 15:18:01,"Jetzt muss ich mal eins fragen . Ich kann doch nicht mit Wlan to go alle Hotspots der Telekom kostenfrei nutzen , oder ? Was ich weiss nur die Hotspots der teilnehmenden Partner / Telekomkunden der Fa . Fon",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141461895900905&cpid=&aid=1096037533756491&vid=2&wid=122768271103611,127469.0,NaN,a255ed96-818f-42b2-84f1-0c0c7e2581da,30745045,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
7,NaN,NaN,www.facebook.com,4,24.09.2016 15:19:10,ich weis nicht wie es geht :(,https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141462655900829&cpid=1141448189235609_1141458792567882&aid=992943974115174&vid=2&wid=122768271103611,7707299.0,NaN,53632505-a61d-423e-bfc2-9f2047ad75b1,30745129,0be928df-7378-421e-be6e-

In [20]:
pd.unique(raw_df.SichterName)

array([nan, 'Hotspot', 'Festnetz', 'Mobilfunk', 'GK', 'MMS', 'A-Team',
       'Mobile'], dtype=object)

In [21]:
df = pd.concat([raw_df[raw_df.Tag.isnull()], raw_df[raw_df.Tag=='Prio-Fall']])
df = df[df.kind == 1]

In [22]:
import re 
#standard regular expressions
GRUBER_URLINTEXT_PAT = re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019]))')
NUMBER_CLEAN = re.compile(r'\d+(:?[.,]?\d+)*')
TIME_CLEAN = re.compile(r'\d+:\d+(?:\s?[u]hr)?')
PUNCT_CLEAN = re.compile(r'(\s?)[\.\,()\[\]\{\}\^\:\'\"\/;…/%//“=–><&]+(\s?)')
DASH_CLEAN = re.compile(r'([^et])[-]')
TWEET_CLEAN = re.compile(r'@\w+')
HASH_CLEAN = re.compile(r'#(\w+)')
SPACE_CLEAN = re.compile(r'(\s)\s+')

def preprocessing(text):
    #a function that cleans strings by removing regular expression from them. Becasue these are tweet data, I used some general
    #strings that can be simply matched and splitted results can be stored as text which is all words.
    try:
        #all lower
        text = text.lower()
        # clean-up url
        text = GRUBER_URLINTEXT_PAT.sub('', text)
        # remove time
        text = TIME_CLEAN.sub('', text)
        # clean-up numbers
        text = NUMBER_CLEAN.sub('', text)
        #punctuation marks are cleaned
        text = PUNCT_CLEAN.sub(r'\1\2',text)
        #clean tagging in tweets
        text = TWEET_CLEAN.sub('', text)
        text = HASH_CLEAN.sub(r'\1', text)
        text = DASH_CLEAN.sub(r'\1 ', text)
        text = SPACE_CLEAN.sub(r'\1', text)
        return ' '.join([word for word in text.split(' ') if word not in stopwords]).strip()
    except:
        return ''
#stop words are words from german language that are also removed from the text of the tweets
with open('./stopwords-de.txt') as f:
    stopwords = f.read().split('\n')

In [23]:
documents = [] # Content
labels = [] # SichterName
tags = [] # Tag

for (idx, row) in tqdm(df.iterrows()):
    labels.append(row.SichterName) # all labels as an array
    documents.append(preprocessing(row.Content))#content of tweets as array
    tags.append(str(row.Tag))#the tag value in a seperate column

55it [00:00, 240.37it/s]


In [24]:
import pickle
# Save processed rows
pickle.dump(documents, open('documents_test.pl', 'wb'))
pickle.dump(labels, open('labels_test.pl', 'wb'))
pickle.dump(tags, open('tags.pl_test', 'wb'))

In [25]:
print(documents)

['hightechno ! ich will claras wg mal so richtig zu elektro dancen sehen', 'muss ich mit hotspot registrieren wlan to go auf meinem handy zu gehen weiß ich nicht wie es zu tun ich bin telekom kunde', 'rt oneerp supportet by detecon one figure one truth ! telekomwall digitalchange https', 'hallo zusammen gab es vor kurzem ein update des mr ? heute ist mir aufgefallen dass einige serienaufnahmen nicht mehr aufgenommen wurden z b rtl unter uns in der liste geplante aufnahme war es vorhanden jedoch mit in der klammer ich musste erst die serienaufnahme löschen und neu programmieren gibt es inzwischen eine lösung wie man den receiver dazu bringt die serien nur einmal aufzunehmen und nicht mit allen wiederholungen ich meine die programmierung über epg den workaround über manuelle aufnahme kenne ich danke und schöne grüße an die kollegen !', 'hallo zusammen gab es vor kurzem ein update des mr ? heute ist mir aufgefallen dass einige serienaufnahmen nicht mehr aufgenommen wurden z b rtl unter un